In [6]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama

base_url="http://localhost:11434"
model="llama3.2"

llm = ChatOllama(
    base_url = base_url,
    model = model,
)



In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

user_prompt = """ Given the user review below, classify the sentiment of the review as positive, negative.
                  Do not consider the sentiment of the user review as neutral.
                  Do not respond with more than one word.
                  Review: {review}"""

user_prompt_template = ChatPromptTemplate.from_template(user_prompt)

chain_user_prompt = user_prompt_template | llm | StrOutputParser()

In [8]:
review = input("Please enter your review: ")
chain_user_prompt.invoke(review)

"I'm ready when you are. Please go ahead and provide the review."

In [4]:
if_postive_review = """ You are an expert in writing replies for positive reviews. 
                        You also need to encourage the user to leave a review on the platform or on social media.
                        The user review is: {review}"""

if_postive_review_template = ChatPromptTemplate.from_template(if_postive_review)
if_postive_review_chain = if_postive_review_template | llm | StrOutputParser()

In [5]:
if_negative_review = """ You are an expert in writing replies for negative reviews.
                         You need to fisrt apologize for the user's bad experience and then offer a solution according to the user review.
                        You also need to encourage the user to share their concern on the following email: 'khanarham0029@gmail.com'.
                        The user review is: {review}"""

if_negative_review_template = ChatPromptTemplate.from_template(if_negative_review)
if_negative_review_chain = if_negative_review_template | llm | StrOutputParser()


In [9]:
def route(info):
    if 'positive' in info['sentiment'].lower():
        return if_postive_review_chain
    else:
        return if_negative_review_chain

In [11]:
from langchain_core.runnables import RunnableLambda

full_chain = {'sentiment':chain_user_prompt, 'review' : lambda x:x['review']}  | RunnableLambda(route)
full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template=' Given the user review below, classify the sentiment of the review as positive, negative.\n                  Do not consider the sentiment of the user review as neutral.\n                  Do not respond with more than one word.\n                  Review: {review}'), additional_kwargs={})])
             | ChatOllama(model='llama3.2', base_url='http://localhost:11434')
             | StrOutputParser(),
  review: RunnableLambda(lambda x: x['review'])
}
| RunnableLambda(route)

In [13]:
review = input("Please enter your review: ")
output=full_chain.invoke({'review':review})
print(output)

Wow, thank you so much for taking the time to share your thoughts about our product! We're thrilled to hear that you were initially skeptical but ended up being pleasantly surprised by its unique charm.

We appreciate your honesty about our design and functionality, which may not be everyone's cup of tea. However, we're glad to hear that it meets its claims most of the time, and that the battery life has a certain "charm" to it – who needs reliability when you can have excitement, right?

Regarding the software, we acknowledge that it has some quirks, but we're working hard to iron out those issues. We appreciate your patience and willingness to give us feedback.

We love that you've decided to recommend our product to those who enjoy a bit of unpredictability in their life! It's definitely not for everyone, but for those who are adventurous and open to trying new things, it can be a game-changer.

If you're considering purchasing or have already purchased our product, we'd love to hea